Ćwiczenie wstępne z MPI. Na podstawie:
* http://selkie.macalester.edu/DistributedPython/index.html

* https://mpi4py.readthedocs.io/en/stable/index.html

In [1]:
!pip install mpi4py

/bin/bash: line 1: pip: command not found


In [2]:
%%writefile example0.py
# -*- coding: utf-8 -*-

from mpi4py import MPI

def main():
  comm = MPI.COMM_WORLD # komunikator zawierający wszystkie procesy
  id = comm.Get_rank() # id aktualnego procesu
  numProc = comm.Get_size() # liczba procesów w komunikatorze
  hostName = MPI.Get_processor_name() # nazwa maszyny na której uruchamiany jest program

  print(f"Jestem procesem nr {id} z {numProc} uruchamianym na maszynie {hostName}")

main()


Overwriting example0.py


In [11]:
#uruchomienie na komputrze lokalnym:
# mpirun -np 4 python example0.py 
# ! oraz opcja --allow-run-as-root dodane na potrzeby notatnika
!mpirun.mpich  -np 4 python3 example0.py

Jestem procesem nr 0 z 1 uruchamianym na maszynie DESKTOP-6S2SSNU
Jestem procesem nr 0 z 1 uruchamianym na maszynie DESKTOP-6S2SSNU
Jestem procesem nr 0 z 1 uruchamianym na maszynie DESKTOP-6S2SSNU
Jestem procesem nr 0 z 1 uruchamianym na maszynie DESKTOP-6S2SSNU


In [1]:
%%writefile example1.py

from mpi4py import MPI

def main():
  comm = MPI.COMM_WORLD # komunikator zawierający wszystkie procesy
  id = comm.Get_rank() # id aktualnego procesu
  numProc = comm.Get_size() # liczba procesów w komunikatorze
  hostName = MPI.Get_processor_name() # nazwa maszyny na której uruchamiany jest program

  if id == 0:
    print(f"Jestem procesem ** głównym ** o id {id} uruchamianym na maszynie {hostName}")
  else:
    print(f"Jestem procesem nr {id} z {numProc} uruchamianym na maszynie {hostName}")


main()

Writing example1.py


In [2]:
!mpirun.mpich  -np 4 python3 example1.py

Jestem procesem ** głównym ** o id 0 uruchamianym na maszynie DESKTOP-6S2SSNU
Jestem procesem ** głównym ** o id 0 uruchamianym na maszynie DESKTOP-6S2SSNU
Jestem procesem ** głównym ** o id 0 uruchamianym na maszynie DESKTOP-6S2SSNU
Jestem procesem ** głównym ** o id 0 uruchamianym na maszynie DESKTOP-6S2SSNU


In [4]:
%%writefile example2.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            # numer procesu
    numProcesses = comm.Get_size()  # łączna liczba procesów
    myHostName = MPI.Get_processor_name()  # nazwa maszyny

    REPS = 16

    if ((REPS % numProcesses) == 0 and numProcesses <= REPS):
        # określenie zakresu iteracji dla danego procesu
        chunkSize = int(REPS / numProcesses)
        start = id * chunkSize
        stop = start + chunkSize

        # wykonanie przypisanych iteracji
        for i in range(start, stop):
            print("On {}: Process {} is performing iteration {}"
                  .format(myHostName, id, i))
    else:
        # jeżeli nie da się podzielić iteracji równo
        if id == 0:
            print("Please run with number of processes divisible by "
                  "and less than or equal to {}.".format(REPS))

if __name__ == "__main__":
    main()


Overwriting example2.py


In [28]:
!mpirun.mpich  -np 2 python3 example2.py

On DESKTOP-6S2SSNU: Process 0 is performing iteration 0
On DESKTOP-6S2SSNU: Process 0 is performing iteration 1
On DESKTOP-6S2SSNU: Process 0 is performing iteration 2
On DESKTOP-6S2SSNU: Process 0 is performing iteration 3
On DESKTOP-6S2SSNU: Process 0 is performing iteration 4
On DESKTOP-6S2SSNU: Process 0 is performing iteration 5
On DESKTOP-6S2SSNU: Process 0 is performing iteration 6
On DESKTOP-6S2SSNU: Process 0 is performing iteration 7
On DESKTOP-6S2SSNU: Process 0 is performing iteration 0
On DESKTOP-6S2SSNU: Process 0 is performing iteration 1
On DESKTOP-6S2SSNU: Process 0 is performing iteration 2
On DESKTOP-6S2SSNU: Process 0 is performing iteration 3
On DESKTOP-6S2SSNU: Process 0 is performing iteration 4
On DESKTOP-6S2SSNU: Process 0 is performing iteration 5
On DESKTOP-6S2SSNU: Process 0 is performing iteration 6
On DESKTOP-6S2SSNU: Process 0 is performing iteration 7



*   Proszę uruchomić powyższy program użwając różnej liczby procesów (N = 1,2,4,8);  

*   zmienić wartość REPS na 16
* wyjaśnić jak schemat dzieli iteracje pętli między procesy

In [2]:
%%writefile example3.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    REPS = 16

    if (numProcesses <= REPS):

        for i in range(id, REPS, numProcesses):
            print("On {}: Process {} is performing iteration {}"\
            .format(myHostName, id, i))

    else:
        # can't have more processes than work; one process reports the error
        if id == 0 :
            print("Please run with number of processes less than \
or equal to {}.".format(REPS))

########## Run the main function
main()


Overwriting example3.py


In [ ]:
!mpirun.mpich  -np 4 python3 example3.py

* Proszę uruchomić powyższy program użwając różnej liczby procesów (N = 1,2,4,8);

* zmienić wartość REPS na 16

* wyjaśnić jak schemat dzieli iteracje pętli między procesy

Komunikacja punkt-punkt

In [3]:
%%writefile example4.py
from mpi4py import MPI

# function to return whether a number of a process is odd or even
def odd(number):
    if (number % 2) == 0:
        return False
    else :
        return True

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if numProcesses > 1 and not odd(numProcesses):
        #generate a list of 8 numbers, beginning with my id
        sendList = list(range(id, id+8))
        if odd(id):
            #odd processes send to their 'left neighbor', then receive from
            comm.send(sendList, dest=id-1)
            receivedList = comm.recv(source=id-1)
        else :
            #even processes receive from their 'right neighbor', then send
            receivedList = comm.recv(source=id+1)
            comm.send(sendList, dest=id+1)

        print("Process {} of {} on {} computed {} and received {}"\
        .format(id, numProcesses, myHostName, sendList, receivedList))

    else :
        if id == 0:
            print("Please run this program with the number of processes \
positive and even")

########## Run the main function
main()

Writing example4.py


In [ ]:
!mpirun.mpich  -np 4 python3 example4.py

Proszę narysować schemat komunikacji pomiędzy poszczególnymi procesami w zadaniu numer 4

##Zadanie
**Zadanie.** Proszę napisać program który generuje tablicę tablic liczb w procesie głównym i rozsyłą je do pozostałych procesów. Procesy odbierają swoją część tablicy i wyliczają oraz wypisują jej sumę;   

In [21]:
%%writefile zad.py
from mpi4py import MPI
from random import randint

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()   # numer procesu
    size = comm.Get_size()   # łączna liczba procesów

    if rank == 0:
        data = []
        # Tworzymy tablice tylko dla procesów od 1 do size-1
        for x in range(1, size):
            subList = []
            for y in range(size):
                subList.append(randint(0, y + x))
            data.append(subList)
        
        # Wysyłamy odpowiednią tablicę do każdego procesu od 1 do size-1
        for x in range(1, size):
            # Dla procesu x dane przechowujemy w data[x-1]
            comm.send(data[x-1], dest=x)
    else:
        recvList = comm.recv(source=0)
        print(f"For process {rank}, sum is {sum(recvList)}, list rec: {recvList}")

if __name__ == "__main__":
    main()


Overwriting zad.py


In [16]:
!mpirun.mpich  -np 4 python3 zad.py

0
0
0
0


Komunikacja kolektywna

In [22]:
%%writefile broadcast_example.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if numProcesses > 1 :

        if id == 0:        # master
            #master: generate a dictionary with arbitrary data in it
            data = list(range(numProcesses))
            print("Master Process {} of {} on {} broadcasts {}"\
            .format(id, numProcesses, myHostName, data))

        else :
            # worker: start with empty data
            data = []
            print("Worker Process {} of {} on {} starts with {}"\
            .format(id, numProcesses, myHostName, data))

        #initiate and complete the broadcast
        data = comm.bcast(data, root=0)

        #check the result
        print("Process {} of {} on {} has {} after the broadcast"\
        .format(id, numProcesses, myHostName, data))

    else :
        print("Please run this program with the number of processes greater than 1")

########## Run the main function
main()


Writing broadcast_example.py


In [ ]:
!mpirun.mpich  -np 4 python3 broadcast_example.py

Scatter, Gather 

In [23]:
%%writefile 14scatter.py
from mpi4py import MPI

# Create a list of lists to be scattered.
def genListOfLists(numElements):
    data = [[0]*3 for i in range(numElements)]
    for i in range(numElements):
        #make small lists of 3 distinct elements
        smallerList = []
        for j in range(1,4):
            smallerList = smallerList + [(i+1)*j]
        # place the small list in the larger list
        data[i] = smallerList
    return data

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    # in mpi4py, the lowercase scatter method only works on lists whose size
    # is the total number of processes.
    numElements = numProcesses      #total elements in list created by master process

    # however, the list can contain lists, like this list of 3-element lists,
    # for example this list of four 3-element lists:
    #     [[1, 2, 3], [2, 4, 6], [3, 6, 9], [4, 8, 12]]

    if id == 0:
        data = genListOfLists(numElements)
        print("Master {} of {} on {} has created list: {}"\
        .format(id, numProcesses, myHostName, data))
    else:
        data = None
        print("Worker Process {} of {} on {} starts with {}"\
        .format(id, numProcesses, myHostName, data))

    #scatter one small list in the large list on node 0 to each of the processes
    result = comm.scatter(data, root=0)

    print("Process {} of {} on {} has result after scatter {}"\
    .format(id, numProcesses, myHostName, result))

    if id == 0:
        print("Master {} of {} on {} has original list after scatter: {}"\
        .format(id, numProcesses, myHostName, data))

########## Run the main function
main()

Writing 14scatter.py


In [ ]:
!mpirun.mpich  -np 4 python3 14scatter.py

Zadanie. Proszę napisać program który generuje tablicę tablic liczb w procesie głównym i rozsyłą je do pozostałych procesów. Procesy odbierają swoją część tablicy i wyliczają oraz wypisują jej sumę. Proszę napisac program z wykorzystaniem komunikacji kolektywnej

In [30]:
%%writefile zad2.py
from mpi4py import MPI
from random import randint

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()   # numer procesu
    size = comm.Get_size()   # łączna liczba procesów
    data = []
    if rank == 0:
        for x in range(size):
            subList = []
            for y in range(size):
                subList.append(randint(0, y + x))
            data.append(subList)
    else:
        list = None
        print(f"Befor for procces {rank}, list is: {list}")    
    # Brodkast tablicy
    list = comm.scatter(data, root=0)

    
    print(f"For process {rank}, sum is {sum(list)}, list rec: {list}")    


if __name__ == "__main__":
    main()


Overwriting zad2.py
